In [12]:
with open('schedule.json','r', encoding='utf-8') as f:
    data = f.read().strip()

In [14]:
def json_to_xml_replace(data: str) -> str:
    
    tab_cnt = 0
    xml = ''
    key = 'root'
    dict_stack = []
    list_stack = [key]
    for line in data.splitlines():
        striped_line = line.strip()
        if striped_line=='': continue
        if striped_line[-1]==',': striped_line = striped_line[:-1]
        
        key, obj = get_token(striped_line, list_stack)
        
        if obj[-1]=='{': 
            xml += tab_cnt * '    ' + f'<{key}>\n'
            dict_stack.append(key)
            tab_cnt+=1
        elif obj[-1] == '[':
            list_stack.append(key)
        elif obj == ']':
            list_stack.pop()
        elif obj == '}':
            dict_key = dict_stack.pop()
            tab_cnt-=1
            xml += tab_cnt * '    ' + f'</{dict_key}>\n'
        else:
            xml += tab_cnt * '    ' + f'<{key}>{obj}</{key}>\n'
    return xml



def get_token(line:str, lst: list) :
    splitted = line.split(':')
    splitted = [split.replace('"', '') for split in splitted]
    if len(splitted)>1:
        splitted = [splitted[0], ':'.join(splitted[1:])]
    if len(splitted)==1: 
        splitted.append(lst[-1])
        splitted = splitted[::-1]
    return  splitted[0], splitted[-1].strip()

d = json_to_xml_replace(data)
print(d)

<root>
    <even_week>false</even_week>
    <date>12.11.2024</date>
    <day_of_week>Tuesday</day_of_week>
    <couples>
        <format>Очно - дистанционный</format>
        <group>2.3</group>
        <type>Практика</type>
        <name>Математический анализ</name>
        <teacher>Юрова Татьяна Сергеевна</teacher>
        <place>
            <campus>Кронверкский пр., д.49, лит.А</campus>
            <audience>2414</audience>
        </place>
        <time>
            <time_from>8:20</time_from>
            <time_to>9:50</time_to>
        </time>
    </couples>
    <couples>
        <format>Очно - дистанционный</format>
        <group>2.1</group>
        <type>Практика</type>
        <name>Линейная алгебра</name>
        <teacher>Ефремов Илья Дмитриевич</teacher>
        <place>
            <campus>Кронверкский пр., д.49, лит.А</campus>
            <audience>1316</audience>
        </place>
        <time>
            <time_from>11:40</time_from>
            <time_to>13:10</time_to>
 

In [16]:
import json
import xmltodict
with open('schedule.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
def bibl(d):
    
    d = xmltodict.unparse({'root': data})
    return d
print(bibl(data))


<?xml version="1.0" encoding="utf-8"?>
<root><even_week>false</even_week><date>12.11.2024</date><day_of_week>Tuesday</day_of_week><couples><format>Очно - дистанционный</format><group>2.3</group><type>Практика</type><name>Математический анализ</name><teacher>Юрова Татьяна Сергеевна</teacher><place><campus>Кронверкский пр., д.49, лит.А</campus><audience>2414</audience></place><time><time_from>8:20</time_from><time_to>9:50</time_to></time></couples><couples><format>Очно - дистанционный</format><group>2.1</group><type>Практика</type><name>Линейная алгебра</name><teacher>Ефремов Илья Дмитриевич</teacher><place><campus>Кронверкский пр., д.49, лит.А</campus><audience>1316</audience></place><time><time_from>11:40</time_from><time_to>13:10</time_to></time></couples></root>


In [18]:
with open('schedule.json', 'r', encoding='utf-8') as f:
    data = f.read().strip()


import re
def regex(data) -> str:
    data = re.sub(r'\"([^\"]+)\"\s*:\s*\"([^\"]+)\"', r'<\1>\2</\1>', data)
    data = re.sub(r'\"([^\"]+)\"\s*:\s*(\d+\.?\d*)', r'<\1>\2</\1>', data)
    data = re.sub(r'\"([^\"]+)\"\s*:\s*(false|true)', r'<\1>\2</\1>', data)
    data = re.sub(r'(?P<spaces> *)\"(\w+)\":\s*{([^{]*)(?P=spaces)}', r'\1<\2>\3\1</\2>', data)
    list_items = re.search(r'( +)\"([^\"]+)\"\s*:\s*\[', data)
    list_key = list_items.group(2)
    spaces = list_items.group(1)
    data = re.sub(r'( +)\"([^\"]+)\"\s*:\s*\[\n([^\]]+)\n\s*\]', r'\3', data)
    data = re.sub(rf'{re.escape(spaces)}\{{([^\}}]+{re.escape(spaces)})\}}', rf'{spaces}<{list_key}>\1</{list_key}>', data)
    data = re.sub(rf'{re.escape(spaces)}\"([^\"]*)\"', rf'{spaces}<{list_key}>\1</{list_key}>', data)
    data = re.sub(rf'{re.escape(spaces)}(\d+\.?\d*|(?:false|true))', rf'{spaces}<{list_key}>\1</{list_key}>', data)
    data = re.sub(r'{([^\}]*)}', r'<root>\1</root>', data)
    data = data.replace(',\n', '\n')
    return data
d = regex(data)
print(d)

<root>
    <even_week>false</even_week>
    <date>12.11.2024</date>
    <day_of_week>Tuesday</day_of_week>
        <couples>
            <format>Очно - дистанционный</format>
            <group>2.3</group>
            <type>Практика</type>
            <name>Математический анализ</name>
            <teacher>Юрова Татьяна Сергеевна</teacher>
            <place>
                <campus>Кронверкский пр., д.49, лит.А</campus>
                <audience>2414</audience>
            </place>
            
            <time>
                <time_from>8:20</time_from>
                <time_to>9:50</time_to>
            </time>
        </couples>

        <couples>
            <format>Очно - дистанционный</format>
            <group>2.1</group>
            <type>Практика</type>
            <name>Линейная алгебра</name>
            <teacher>Ефремов Илья Дмитриевич</teacher>
            <place>
                <campus>Кронверкский пр., д.49, лит.А</campus>
                <audience>1316</audience>
 

In [9]:
def parse_json(data):
    di={}
    dat=data[1:-1].strip()
    skobki=0
    arrcheck=0
    is_key=True
    tekkey=''
    tekznach=''
    for i in range(len(dat)) :
        if dat[i]=='{':
            skobki+=1
        elif dat[i]=='[':
            arrcheck+=1
        elif dat[i]==']':
            arrcheck-=1
        elif dat[i]=='}':
            skobki-=1
        elif dat[i]==':' and skobki==0:
            is_key=False
            continue
        elif dat[i]==',' and skobki==0 and arrcheck==0:
            tekznach=tekznach.replace('\n','')
            di[tekkey.strip().strip('\n')]=tekznach
            tekkey=''
            tekznach=''
            is_key=True
            continue
        if is_key==True:
            tekkey+=dat[i]
        else:
            tekznach+=dat[i]
    if tekznach and tekkey:
        tekznach = tekznach.replace('\n', '')
        tekznach = tekznach.replace(' ', '')
        di[tekkey]=tekznach
    return di


def to_dict(dii):
    din={}
    for key, value in dii.items():
        try:
            din[key.strip()[1:-1]]=eval(value)
        except:
            din[key.strip()[1:-1]] = value
            continue
    return din


def to_string(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()
        return content
def to_xml(json_data, count):
    xml_result = ""
    probels = "  " * count
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            if isinstance(value, (dict, list)):
                xml_result += f"{probels}<{key}>\n{to_xml(value, count + 1)}{probels}</{key}>\n"
            else:
                xml_result += f"{probels}<{key}>{value}</{key}>\n"
    elif isinstance(json_data, list):
        for item in json_data:
            xml_result += f"{probels}<item>\n{to_xml(item, count + 1)}{probels}</item>\n"
    else:
        xml_result += f"{probels}{json_data}\n"

    return xml_result
data=to_string("schedule.json")
di=parse_json(data)

di=to_dict(di)
d='''<root>\n''' + to_xml(di,0) +'''</root>'''
print(d)
'''
with open('xml_output.xml', 'w', encoding='utf-8') as file:
    file.write(xml_output)'''


<root>
<even_week> false</even_week>
<date>12.11.2024</date>
<day_of_week>Tuesday</day_of_week>
<couples>
  <item>
    <format>Очно-дистанционный</format>
    <group>2.3</group>
    <type>Практика</type>
    <name>Математическийанализ</name>
    <teacher>ЮроваТатьянаСергеевна</teacher>
    <place>
      <campus>Кронверкскийпр.,д.49,лит.А</campus>
      <audience>2414</audience>
    </place>
    <time>
      <time_from>8:20</time_from>
      <time_to>9:50</time_to>
    </time>
  </item>
  <item>
    <format>Очно-дистанционный</format>
    <group>2.1</group>
    <type>Практика</type>
    <name>Линейнаяалгебра</name>
    <teacher>ЕфремовИльяДмитриевич</teacher>
    <place>
      <campus>Кронверкскийпр.,д.49,лит.А</campus>
      <audience>1316</audience>
    </place>
    <time>
      <time_from>11:40</time_from>
      <time_to>13:10</time_to>
    </time>
  </item>
</couples>
</root>


"\nwith open('xml_output.xml', 'w', encoding='utf-8') as file:\n    file.write(xml_output)"

In [ ]:
with open('schedule.json', 'r', encoding='utf-8') as file:
        content = file.read()
d=content
%timeit for i in range(100): json_to_xml_replace(d)
%timeit for i in range(100): bibl(d)
%timeit for i in range(100): regex(d)
%timeit for i in range(100): to_xml(to_dict(parse_json(d)),0)

9.44 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.29 ms ± 39.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
17.3 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
69.3 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [29]:
import pandas as pd
with open('schedule.json', 'r', encoding='utf-8') as file:
        content = json.load(file)
d=content
print(d)
df=pd.DataFrame(d)
df

{'even_week': False, 'date': '12.11.2024', 'day_of_week': 'Tuesday', 'couples': [{'format': 'Очно - дистанционный', 'group': '2.3', 'type': 'Практика', 'name': 'Математический анализ', 'teacher': 'Юрова Татьяна Сергеевна', 'place': {'campus': 'Кронверкский пр., д.49, лит.А', 'audience': 2414}, 'time': {'time_from': '8:20', 'time_to': '9:50'}}, {'format': 'Очно - дистанционный', 'group': '2.1', 'type': 'Практика', 'name': 'Линейная алгебра', 'teacher': 'Ефремов Илья Дмитриевич', 'place': {'campus': 'Кронверкский пр., д.49, лит.А', 'audience': 1316}, 'time': {'time_from': '11:40', 'time_to': '13:10'}}]}


,even_week,date,day_of_week,couples
0,False,12.11.2024,Tuesday,"{'format': 'Очно - дистанционный', 'group': '2..."
1,False,12.11.2024,Tuesday,"{'format': 'Очно - дистанционный', 'group': '2..."
